# Auditing_Data_Quality

## Project :
The task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).


In [3]:

import csv
import pprint

INPUT_FILE = 'data/autos1.csv'
OUTPUT_GOOD = 'data/autos-valid.csv'
OUTPUT_BAD = 'data/autos-Fixme.csv'


            
            
def process_file(input_file, output_good, output_bad):
    # store data into lists for output
    data_good = []
    data_bad = []
    with open(input_file, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        for row in reader:
            # validate URI value
            if row['URI'].find("dbpedia.org") < 0:
                continue

            ps_year = row['productionStartYear'][:4]
            try: # use try/except to filter valid items
                ps_year = int(ps_year)
                row['productionStartYear'] = ps_year
                if (ps_year >= 1886) and (ps_year <= 2014):
                    data_good.append(row)
                else:
                    data_bad.append(row)
            except ValueError: # non-numeric strings caught by exception
                if ps_year == 'NULL':
                    data_bad.append(row)

    # Write processed data to output files
    with open(output_good, "w", encoding="utf-8") as good:
        writer = csv.DictWriter(good, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_good:
            writer.writerow(row)

    with open(output_bad, "w", encoding="utf-8") as bad:
        writer = csv.DictWriter(bad, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in data_bad:
            writer.writerow(row)
            

def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

## Project :

In this problem set we work with cities infobox data, audit it, come up with a cleaning idea and then clean it up.
In the first exercise we want you to audit the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- 'NoneType' if the value is a string "NULL" or an empty string ""
- 'list', if the value starts with "{"
- 'int', if the value can be cast to int
- 'float', if the value can be cast to float, but is not an int
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and the datatypes that can be found in the field.
All the data initially is a string, so you have to do some checks on the values first.


In [28]:
import codecs
import csv
import json
import pprint

CITIES = 'data/cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label", "isPartOf_label", "areaCode", "populationTotal", 
          "elevation", "maximumElevation", "minimumElevation", "populationDensity", "wgs84_pos#lat", "wgs84_pos#long", 
          "areaLand", "areaMetro", "areaUrban"]

def is_int(value):
    try:
        int(value)
        return True
    except:
        return False

def is_float(value):
    try:
        float(value)
        return True
    except:
        return False

def audit_file(filename, fields):
    fieldtypes = {}

    for field in fields:
        fieldtypes[field] = set()

    with open(filename, 'r') as input:
        reader = csv.DictReader(input)

        for row in reader:
            if row['URI'].find('dbpedia') > -1:
                
                for field in fields:

                    if row[field] == "NULL" or row[field] == "":
                        fieldtypes[field].add(type(None))
                    elif row[field][0] == "{":
                        fieldtypes[field].add(type([]))
                    elif is_int(row[field]):
                        fieldtypes[field].add(type(1))    
                    elif is_float(row[field]):
                        fieldtypes[field].add(type(1.1))
                    else:
                        fieldtypes[field].add(type(''))  

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)
    
    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
    
if __name__ == "__main__":
    test()

{'areaCode': {<class 'int'>, <class 'NoneType'>, <class 'str'>},
 'areaLand': {<class 'list'>, <class 'NoneType'>, <class 'float'>},
 'areaMetro': {<class 'NoneType'>, <class 'float'>},
 'areaUrban': {<class 'NoneType'>, <class 'float'>},
 'elevation': {<class 'list'>, <class 'NoneType'>, <class 'float'>},
 'governmentType_label': {<class 'NoneType'>, <class 'str'>},
 'homepage': {<class 'NoneType'>, <class 'str'>},
 'isPartOf_label': {<class 'list'>, <class 'str'>, <class 'NoneType'>},
 'maximumElevation': {<class 'NoneType'>},
 'minimumElevation': {<class 'NoneType'>},
 'name': {<class 'list'>, <class 'str'>, <class 'NoneType'>},
 'populationDensity': {<class 'list'>, <class 'NoneType'>, <class 'float'>},
 'populationTotal': {<class 'int'>, <class 'NoneType'>},
 'timeZone_label': {<class 'str'>, <class 'NoneType'>},
 'utcOffset': {<class 'list'>,
               <class 'int'>,
               <class 'NoneType'>,
               <class 'str'>},
 'wgs84_pos#lat': {<class 'float'>},
 'wgs8